In [12]:
import pandas as pd

In [13]:
import pandas as pd

# Read the Excel file as strings to avoid DateParseError
#df = pd.read_excel('Train/Belt 1 9 Months negative data.xlsx', dtype=str)
df = pd.read_excel('Train/Belt 1 9 Months negative data.xlsx')
# Convert 'Timestamp' to datetime, invalid dates will be set to NaT
df['Timestamp'] = pd.to_datetime(df['Timestamp'], errors='coerce') 

# Drop rows where the 'Timestamp' is NaT (invalid date)
df.dropna(subset=['Timestamp'], inplace=True)


# Check initial NaT count in the Timestamp column (this counts blank/missing values)
print("Initial NaT count:", df['Timestamp'].isna().sum())

train_cols = [
    "Vibration Frequency",
    "Vibration Amplitude",
    "Bearing Temperature",
    "Motor Temperature",
    "Belt Load",
    "Torque",
    "Noise Levels",
    "Current and Voltage",
    "Hydraulic Pressure",
    "Belt Thickness",
    "Roller Condition",  # Removed due to -1 correlation
]

# Convert train_cols from string to float safely
#df[train_cols] = df[train_cols].apply(pd.to_numeric, errors='coerce')

# Replace invalid date entry in Timestamp before converting
#df['Timestamp'] = df['Timestamp'].str.replace('2/29/2026', '3/1/2026', regex=False)

# Convert Timestamp column to datetime, invalid parses become NaT
#df['Timestamp'] = pd.to_datetime(df['Timestamp'], errors='coerce')

# Shift only the valid timestamps (ignore NaT) that are on or after 2026-03-01
#df.loc[df['Timestamp'].notna() & (df['Timestamp'] >= pd.Timestamp('2026-03-01')), 'Timestamp'] += pd.Timedelta(days=1)


Initial NaT count: 0


In [14]:
print(df['Timestamp'].is_monotonic_increasing)
print("NaT count:", df['Timestamp'].isna().sum())

True
NaT count: 0


In [15]:
df

,Name,Timestamp,Status,Description,Vibration Frequency,Vibration Amplitude,Bearing Temperature,Motor Temperature,Belt Load,Torque,Noise Levels,Current and Voltage,Hydraulic Pressure,Belt Thickness,Roller Condition
0,Conveyor Belt 4,2025-08-15 00:00:00,Running,NaN,1490.82,0.04,77.130,96.902,1.36,318.07,55.12,15.79,382.09,1.58000,86.000
1,Conveyor Belt 4,2025-08-15 00:15:00,Running,NaN,1498.37,0.04,77.076,96.975,1.07,295.50,59.68,14.34,376.48,1.57950,85.854
2,Conveyor Belt 4,2025-08-15 00:30:00,Running,NaN,1503.22,0.06,77.307,96.755,1.21,314.38,58.20,15.03,384.20,1.57925,85.781
3,Conveyor Belt 4,2025-08-15 00:45:00,Running,NaN,1508.11,0.04,77.474,97.661,1.29,311.84,56.16,15.43,379.79,1.57900,85.708
4,Conveyor Belt 4,2025-08-15 01:00:00,Running,NaN,1498.13,0.06,77.785,97.471,1.07,317.14,55.39,14.35,383.95,1.57875,85.635
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26204,Conveyor Belt 4,2026-05-13 23:00:00,Running,NaN,1502.84,0.05,76.119,95.287,1.39,305.42,62.72,15.93,377.00,1.63700,68.610
26205,Conveyor Belt 4,2026-05-13 23:15:00,Running,NaN,1503.36,0.05,75.889,95.888,1.06,312.37,64.70,14.28,377.08,1.63675,68.537
26206,Conveyor Belt 4,2026-05-13 23:30:00,Running,NaN,1498.34,0.04,76.001,95.738,1.28,287.36,57.81,15.42,384.19,1.63650,68.464
26207,Conveyor Belt 4,2026-05-13 23:45:00,Running,NaN,1495.26,0.05,76.106,95.848,1.30,297.35,58.84,15.49,376.62,1.63625,68.391


In [16]:
# Define the expected ranges as a dictionary
expected_ranges_data = {
    'Vibration Frequency': {'Lower Bound': 1490, 'Upper Bound': 1510},
    'Vibration Amplitude': {'Lower Bound': 0.04, 'Upper Bound': 0.06},
    'Bearing Temperature': {'Lower Bound': 60, 'Upper Bound': 80},
    'Motor Temperature': {'Lower Bound': 80, 'Upper Bound': 100},
    'Belt Load': {'Lower Bound': 1.0, 'Upper Bound': 1.4},
    'Torque': {'Lower Bound': 280, 'Upper Bound': 320},
    'Noise Levels': {'Lower Bound': 55, 'Upper Bound': 65},
    'Current and Voltage': {'Lower Bound': 14, 'Upper Bound': 16},
    'Hydraulic Pressure': {'Lower Bound': 375, 'Upper Bound': 385},
    'Belt Thickness': {'Lower Bound': 1.5, 'Upper Bound': 1.7},
    'Roller Condition': {'Lower Bound': 65, 'Upper Bound': 100}
}

# Create a Pandas DataFrame from the dictionary
expected_ranges = pd.DataFrame(expected_ranges_data).T
expected_ranges

,Lower Bound,Upper Bound
Vibration Frequency,1490.00,1510.00
Vibration Amplitude,0.04,0.06
Bearing Temperature,60.00,80.00
Motor Temperature,80.00,100.00
Belt Load,1.00,1.40
Torque,280.00,320.00
Noise Levels,55.00,65.00
Current and Voltage,14.00,16.00
Hydraulic Pressure,375.00,385.00
Belt Thickness,1.50,1.70


In [17]:
import pandas as pd

def filter_out_of_range(df, expected_ranges):
    # Convert relevant columns to numeric
    df[expected_ranges.index] = df[expected_ranges.index].apply(pd.to_numeric, errors='coerce')
    
    # Create a boolean mask that is False by default
    mask = pd.Series(False, index=df.index)

    for column in expected_ranges.index:
        lower_bound = expected_ranges.loc[column, 'Lower Bound']
        upper_bound = expected_ranges.loc[column, 'Upper Bound']

        # Update mask to include rows where values are out of range
        mask |= (df[column] < lower_bound) | (df[column] > upper_bound)

    # Apply the combined mask to filter the DataFrame
    out_of_range_rows = df[mask]

    return out_of_range_rows

# Get rows where any column is outside its expected range
out_of_range_df = filter_out_of_range(df, expected_ranges)

out_of_range_df


,Name,Timestamp,Status,Description,Vibration Frequency,Vibration Amplitude,Bearing Temperature,Motor Temperature,Belt Load,Torque,Noise Levels,Current and Voltage,Hydraulic Pressure,Belt Thickness,Roller Condition
2392,Conveyor Belt 4,2025-09-08 22:00:00,Running,NaN,1520.69,0.05,75.089,94.300,1.16,311.37,65.00,16.79,379.97,1.66025,88.393
2398,Conveyor Belt 4,2025-09-08 23:30:00,Running,NaN,1498.34,0.04,76.001,95.738,1.28,287.36,57.81,16.32,384.19,1.65875,87.955
2399,Conveyor Belt 4,2025-09-08 23:45:00,Running,NaN,1495.26,0.05,76.106,95.848,1.30,297.35,58.84,16.52,376.62,1.65850,87.882
2400,Conveyor Belt 4,2025-09-09 00:00:00,Running,NaN,1487.28,0.04,77.130,96.902,1.36,318.07,55.12,15.93,382.09,1.65825,87.809
2402,Conveyor Belt 4,2025-09-09 00:30:00,Running,NaN,1503.22,0.06,77.307,96.755,1.21,314.38,58.20,16.12,384.20,1.65775,87.663
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18439,Conveyor Belt 4,2026-02-23 01:45:00,Down,Irregular Vibrations,1506.80,0.07,65.444,85.230,1.39,294.49,70.94,15.96,384.44,1.69350,78.611
18440,Conveyor Belt 4,2026-02-23 02:00:00,Down,Irregular Vibrations,1506.80,0.07,65.444,85.230,1.39,294.49,70.94,15.96,384.44,1.69350,78.611
18441,Conveyor Belt 4,2026-02-23 02:15:00,Down,Irregular Vibrations,1506.80,0.07,65.444,85.230,1.39,294.49,70.94,15.96,384.44,1.69350,78.611
18442,Conveyor Belt 4,2026-02-23 02:30:00,Down,Irregular Vibrations,1506.80,0.07,65.444,85.230,1.39,294.49,70.94,15.96,384.44,1.69350,78.611


In [18]:
import pandas as pd

def add_out_of_range_columns(df, expected_ranges):
    # Convert relevant columns to numeric
    df[expected_ranges.index] = df[expected_ranges.index].apply(pd.to_numeric, errors='coerce')

    # Iterate over each column in expected_ranges to check if values are within range
    for column in expected_ranges.index:
        lower_bound = expected_ranges.loc[column, 'Lower Bound']
        upper_bound = expected_ranges.loc[column, 'Upper Bound']

        # Create a new column with 1 if out of range, 0 if within range
        df[column + '_OutOfRange'] = ((df[column] < lower_bound) | (df[column] > upper_bound)).astype(int)

    return df

# Apply the function to add out-of-range columns
df_with_range_flags = add_out_of_range_columns(df, expected_ranges)

# Display the updated DataFrame
df_with_range_flags


,Name,Timestamp,Status,Description,Vibration Frequency,Vibration Amplitude,Bearing Temperature,Motor Temperature,Belt Load,Torque,...,Vibration Amplitude_OutOfRange,Bearing Temperature_OutOfRange,Motor Temperature_OutOfRange,Belt Load_OutOfRange,Torque_OutOfRange,Noise Levels_OutOfRange,Current and Voltage_OutOfRange,Hydraulic Pressure_OutOfRange,Belt Thickness_OutOfRange,Roller Condition_OutOfRange
0,Conveyor Belt 4,2025-08-15 00:00:00,Running,NaN,1490.82,0.04,77.130,96.902,1.36,318.07,...,0,0,0,0,0,0,0,0,0,0
1,Conveyor Belt 4,2025-08-15 00:15:00,Running,NaN,1498.37,0.04,77.076,96.975,1.07,295.50,...,0,0,0,0,0,0,0,0,0,0
2,Conveyor Belt 4,2025-08-15 00:30:00,Running,NaN,1503.22,0.06,77.307,96.755,1.21,314.38,...,0,0,0,0,0,0,0,0,0,0
3,Conveyor Belt 4,2025-08-15 00:45:00,Running,NaN,1508.11,0.04,77.474,97.661,1.29,311.84,...,0,0,0,0,0,0,0,0,0,0
4,Conveyor Belt 4,2025-08-15 01:00:00,Running,NaN,1498.13,0.06,77.785,97.471,1.07,317.14,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26204,Conveyor Belt 4,2026-05-13 23:00:00,Running,NaN,1502.84,0.05,76.119,95.287,1.39,305.42,...,0,0,0,0,0,0,0,0,0,0
26205,Conveyor Belt 4,2026-05-13 23:15:00,Running,NaN,1503.36,0.05,75.889,95.888,1.06,312.37,...,0,0,0,0,0,0,0,0,0,0
26206,Conveyor Belt 4,2026-05-13 23:30:00,Running,NaN,1498.34,0.04,76.001,95.738,1.28,287.36,...,0,0,0,0,0,0,0,0,0,0
26207,Conveyor Belt 4,2026-05-13 23:45:00,Running,NaN,1495.26,0.05,76.106,95.848,1.30,297.35,...,0,0,0,0,0,0,0,0,0,0


In [19]:
# One-hot encode the "Status" column
df_with_range_flags = pd.get_dummies(df_with_range_flags, columns=['Status'], drop_first=False, dtype='int')

# Display the updated DataFrame
df_with_range_flags

,Name,Timestamp,Description,Vibration Frequency,Vibration Amplitude,Bearing Temperature,Motor Temperature,Belt Load,Torque,Noise Levels,...,Belt Load_OutOfRange,Torque_OutOfRange,Noise Levels_OutOfRange,Current and Voltage_OutOfRange,Hydraulic Pressure_OutOfRange,Belt Thickness_OutOfRange,Roller Condition_OutOfRange,Status_Down,Status_Maintenance,Status_Running
0,Conveyor Belt 4,2025-08-15 00:00:00,NaN,1490.82,0.04,77.130,96.902,1.36,318.07,55.12,...,0,0,0,0,0,0,0,0,0,1
1,Conveyor Belt 4,2025-08-15 00:15:00,NaN,1498.37,0.04,77.076,96.975,1.07,295.50,59.68,...,0,0,0,0,0,0,0,0,0,1
2,Conveyor Belt 4,2025-08-15 00:30:00,NaN,1503.22,0.06,77.307,96.755,1.21,314.38,58.20,...,0,0,0,0,0,0,0,0,0,1
3,Conveyor Belt 4,2025-08-15 00:45:00,NaN,1508.11,0.04,77.474,97.661,1.29,311.84,56.16,...,0,0,0,0,0,0,0,0,0,1
4,Conveyor Belt 4,2025-08-15 01:00:00,NaN,1498.13,0.06,77.785,97.471,1.07,317.14,55.39,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26204,Conveyor Belt 4,2026-05-13 23:00:00,NaN,1502.84,0.05,76.119,95.287,1.39,305.42,62.72,...,0,0,0,0,0,0,0,0,0,1
26205,Conveyor Belt 4,2026-05-13 23:15:00,NaN,1503.36,0.05,75.889,95.888,1.06,312.37,64.70,...,0,0,0,0,0,0,0,0,0,1
26206,Conveyor Belt 4,2026-05-13 23:30:00,NaN,1498.34,0.04,76.001,95.738,1.28,287.36,57.81,...,0,0,0,0,0,0,0,0,0,1
26207,Conveyor Belt 4,2026-05-13 23:45:00,NaN,1495.26,0.05,76.106,95.848,1.30,297.35,58.84,...,0,0,0,0,0,0,0,0,0,1


In [20]:
df_with_range_flags.iloc[0]

Name                                  Conveyor Belt 4
Timestamp                         2025-08-15 00:00:00
Description                                       NaN
Vibration Frequency                           1490.82
Vibration Amplitude                              0.04
Bearing Temperature                             77.13
Motor Temperature                              96.902
Belt Load                                        1.36
Torque                                         318.07
Noise Levels                                    55.12
Current and Voltage                             15.79
Hydraulic Pressure                             382.09
Belt Thickness                                   1.58
Roller Condition                                 86.0
Vibration Frequency_OutOfRange                      0
Vibration Amplitude_OutOfRange                      0
Bearing Temperature_OutOfRange                      0
Motor Temperature_OutOfRange                        0
Belt Load_OutOfRange        

In [21]:
#df.iloc[2756]

In [25]:
df_with_range_flags

,Name,Timestamp,Description,Vibration Frequency,Vibration Amplitude,Bearing Temperature,Motor Temperature,Belt Load,Torque,Noise Levels,...,Belt Load_OutOfRange,Torque_OutOfRange,Noise Levels_OutOfRange,Current and Voltage_OutOfRange,Hydraulic Pressure_OutOfRange,Belt Thickness_OutOfRange,Roller Condition_OutOfRange,Status_Down,Status_Maintenance,Status_Running
0,Conveyor Belt 4,2025-08-15 00:00:00,NaN,1490.82,0.04,77.130,96.902,1.36,318.07,55.12,...,0,0,0,0,0,0,0,0,0,1
1,Conveyor Belt 4,2025-08-15 00:15:00,NaN,1498.37,0.04,77.076,96.975,1.07,295.50,59.68,...,0,0,0,0,0,0,0,0,0,1
2,Conveyor Belt 4,2025-08-15 00:30:00,NaN,1503.22,0.06,77.307,96.755,1.21,314.38,58.20,...,0,0,0,0,0,0,0,0,0,1
3,Conveyor Belt 4,2025-08-15 00:45:00,NaN,1508.11,0.04,77.474,97.661,1.29,311.84,56.16,...,0,0,0,0,0,0,0,0,0,1
4,Conveyor Belt 4,2025-08-15 01:00:00,NaN,1498.13,0.06,77.785,97.471,1.07,317.14,55.39,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26204,Conveyor Belt 4,2026-05-13 23:00:00,NaN,1502.84,0.05,76.119,95.287,1.39,305.42,62.72,...,0,0,0,0,0,0,0,0,0,1
26205,Conveyor Belt 4,2026-05-13 23:15:00,NaN,1503.36,0.05,75.889,95.888,1.06,312.37,64.70,...,0,0,0,0,0,0,0,0,0,1
26206,Conveyor Belt 4,2026-05-13 23:30:00,NaN,1498.34,0.04,76.001,95.738,1.28,287.36,57.81,...,0,0,0,0,0,0,0,0,0,1
26207,Conveyor Belt 4,2026-05-13 23:45:00,NaN,1495.26,0.05,76.106,95.848,1.30,297.35,58.84,...,0,0,0,0,0,0,0,0,0,1


In [22]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from datetime import datetime, timedelta

# Load data
#data = pd.read_excel('Train/Belt 1 9 Months negative data.xlsx')

# Convert timestamp column to datetime
#data['Timestamp'] = pd.to_datetime(data['Timestamp'])

# Sort by time (important for time series modeling)
#data = data.sort_values(by='Timestamp').reset_index(drop=True)

# Compute time until next failure
data = df_with_range_flags.copy()
data['time_until_failure'] = np.nan  # Initialize column

# Get indices where system goes down
down_indices = data.index[data['Status_Down'] == 1].tolist()

# Assign "time until failure" to each row
for i in range(len(down_indices) - 1):
    start_idx = down_indices[i]  # Current failure index
    next_idx = down_indices[i + 1]  # Next failure index
    next_failure_time = data.loc[next_idx, 'Timestamp']

    # Assign time difference (in hours) to rows between failures
    data.loc[start_idx:next_idx, 'time_until_failure'] = (
        (next_failure_time - data.loc[start_idx:next_idx, 'Timestamp']).dt.total_seconds() / 3600
    )

# Drop NaN values (these are at the last failure point)
data.dropna(subset=['time_until_failure'], inplace=True)

# Select features for prediction
feature_cols = [
    'Vibration Frequency', 'Vibration Amplitude', 'Bearing Temperature',
    'Motor Temperature', 'Belt Load', 'Torque', 'Noise Levels',
    'Current and Voltage', 'Hydraulic Pressure', 'Belt Thickness', 'Roller Condition',
    'Vibration Frequency_OutOfRange', 'Vibration Amplitude_OutOfRange', 'Bearing Temperature_OutOfRange',
    'Motor Temperature_OutOfRange', 'Belt Load_OutOfRange', 'Torque_OutOfRange',
    'Noise Levels_OutOfRange', 'Current and Voltage_OutOfRange', 'Hydraulic Pressure_OutOfRange',
    'Belt Thickness_OutOfRange', 'Roller Condition_OutOfRange',
    'Status_Running', 'Status_Maintenance'
]

X = data[feature_cols]
y = data['time_until_failure']

# Split into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, test_size=0.2)

# Train an XGBoost regression model
model = xgb.XGBRegressor(objective='reg:squarederror', eval_metric='mae')
model.fit(X_train, y_train)

# Predict time until failure
y_pred = model.predict(X_test)

# Evaluate model performance
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error (MAE): {mae:.2f} hours")


Mean Absolute Error (MAE): 435.76 hours


In [24]:
X_train

,Vibration Frequency,Vibration Amplitude,Bearing Temperature,Motor Temperature,Belt Load,Torque,Noise Levels,Current and Voltage,Hydraulic Pressure,Belt Thickness,...,Motor Temperature_OutOfRange,Belt Load_OutOfRange,Torque_OutOfRange,Noise Levels_OutOfRange,Current and Voltage_OutOfRange,Hydraulic Pressure_OutOfRange,Belt Thickness_OutOfRange,Roller Condition_OutOfRange,Status_Running,Status_Maintenance
2872,1522.69,0.05,75.089,94.300,1.16,311.37,65.00,16.79,379.97,1.54900,...,0,0,0,0,1,0,0,0,0,0
2873,1522.69,0.05,75.089,94.300,1.16,311.37,65.00,16.79,379.97,1.54900,...,0,0,0,0,1,0,0,0,0,0
2874,1522.69,0.05,75.089,94.300,1.16,311.37,65.00,16.79,379.97,1.54900,...,0,0,0,0,1,0,0,0,0,0
2875,1522.69,0.05,75.089,94.300,1.16,311.37,65.00,16.79,379.97,1.54900,...,0,0,0,0,1,0,0,0,0,0
2876,1522.69,0.05,75.089,94.300,1.16,311.37,65.00,16.79,379.97,1.54900,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15324,1505.60,0.04,69.091,88.167,1.30,313.92,60.26,15.52,380.81,1.64500,...,0,0,0,0,0,0,0,0,1,0
15325,1499.18,0.04,68.816,89.137,1.34,299.01,60.49,15.69,377.41,1.64475,...,0,0,0,0,0,0,0,0,1,0
15326,1496.82,0.05,69.964,88.995,1.32,304.50,62.90,15.58,382.91,1.64450,...,0,0,0,0,0,0,0,0,1,0
15327,1496.09,0.06,69.467,89.266,1.16,286.35,56.58,14.82,383.43,1.64425,...,0,0,0,0,0,0,0,0,1,0
